# Connection to Vissim COM Server

In [ ]:
from __future__ import print_function
import os
# COM-Server
import win32com.client as com

Vissim = com.gencache.EnsureDispatch("Vissim.Vissim")

Path_of_COM_Basic_Commands_network = os.getcwd()

# Load the Vissim File and layout

In [785]:
Filename = os.path.join(Path_of_COM_Basic_Commands_network, 'kia_parking.inpx')

flag_read_additionally = False # you can read network(elements) additionally, in this case set "flag_read_additionally" to true
Vissim.LoadNet(Filename, flag_read_additionally)

Filename = os.path.join(Path_of_COM_Basic_Commands_network, 'kia_parking.layx')
Vissim.LoadLayout(Filename)

# Load arrival and departure distributions

In [ ]:
%pip install pandas

In [6]:
import pandas as pd

In [9]:
# Passengers_at_dropoff = pd.read_csv(r'\Datasets\Total_Passengers_departures.csv')
Passengers_at_dropoff = pd.read_csv(r"C:\Users\CiSTUP\OneDrive - Indian Institute of Science\vs_codes\vissim_parking\Datasets\Total_Passengers_departures.csv")

# Passengers_for_pickup = pd.read_csv(r'\Datasets\Total_Passengers_Arrival.csv')
Passengers_for_pickup = pd.read_csv(r"C:\Users\CiSTUP\OneDrive - Indian Institute of Science\vs_codes\vissim_parking\Datasets\Total_Passengers_Arrival.csv")

In [10]:
Passengers_at_dropoff.head()

,Total_Passengers,TIME
0,10,0
1,10,1
2,10,2
3,10,3
4,10,4


In [11]:
Passengers_for_pickup.head()

,Total_Passengers,TIME
0,39,0
1,61,1
2,103,2
3,102,3
4,63,4


# Passenger Time interval setting 

- Passenger arrival distribution is divided into 1 minute  interval from 0 to 87000 seconds (24 hours)
- Initial 10 minutes are not used for producing passengers

In [12]:
TimeIntervalSet = 5
time_interval_set = Vissim.Net.TimeIntervalSets.ItemByKey(TimeIntervalSet)
time_interval = time_interval_set.TimeInts.AddTimeInterval(1)
j = 60
for i in range(0,1448*60,60):

# Set the attributes for the TimeInterval
    time_interval.SetAttValue("Start", j)
    time_interval = time_interval_set.TimeInts.AddTimeInterval(1)

    j += 60

# Passenger input setting

- Passengers are added for every 60 seconds interval according to the passenger arrival distribution and in vissim each input is taken in count/hour so we multiply the passenger arrival distribution by 60 to get the passenger input count/hour

In [19]:
Area1 = Vissim.Net.Areas.ItemByKey(8)  # Dropoff area
Area1.SetAttValue('Name','Passengers Arrivals')
PedInput = Vissim.Net.PedestrianInputs.AddPedestrianInput(0, Area1)
# for i in range(11,17281):
#     PedInput.SetAttValue('Volume(' +str(i)+ ')', random.randint(0, 2160))

In [20]:
for id,row in Passengers_for_pickup.iterrows():
    PedInput.SetAttValue('Volume(' +str(row['TIME'] + 11)+ ')', row['Total_Passengers']*60)

# Vehicle Time interval setting

In [87]:
TimeIntervalSet = 1
time_interval_set = Vissim.Net.TimeIntervalSets.ItemByKey(TimeIntervalSet)
# Create a new TimeInterval object
time_interval = time_interval_set.TimeInts.AddTimeInterval(1)
j = 60
for i in range(0,1448*60,60):

# Set the attributes for the TimeInterval
    time_interval.SetAttValue("Start", j)
    time_interval = time_interval_set.TimeInts.AddTimeInterval(1)

    j += 60

# Vehicle input setting

In [88]:
VehInput = Vissim.Net.VehicleInputs.AddVehicleInput(0, Vissim.Net.Links.ItemByKey(18))
 # unsigned int Key, Link,
VehInput.SetAttValue('Name','Passengers Dropout')

In [28]:
VehInput = Vissim.Net.VehicleInputs.AddVehicleInput(0, Vissim.Net.Links.ItemByKey(18))
 # unsigned int Key, Link,
VehInput.SetAttValue('Name','Passengers Dropout')
# for i in range(1,17281):
#     VehInput.SetAttValue('Volume(' +str(i)+ ')', random.randint(0, 100))
#     VehInput.SetAttValue('VehComp(' +str(i)+ ')',2)

### Assigning volume at Dropout zone

In [89]:
for id,row in Passengers_at_dropoff.iterrows():
    VehInput.SetAttValue('Volume(' +str(row['TIME'] + 11)+ ')', row['Total_Passengers']*60)
    VehInput.SetAttValue('VehComp(' +str(row['TIME'] + 11)+ ')', 2)
    # VehInput.SetAttValue('VehType(' +str(row['TIME'] + 11)+ ')', 'Exact')

### Assigning initial volume at reserved parking zone

In [278]:
initial_volume = 300
initial_volume_BLU = 100

In [124]:
VehInput = Vissim.Net.VehicleInputs.AddVehicleInput(0, Vissim.Net.Links.ItemByKey(68))
VehInput.SetAttValue('Name','OLA Initial Reserved Volume')
for i in range(1,9):
    VehInput.SetAttValue('Volume(' +str(i)+ ')', initial_volume)
    VehInput.SetAttValue('VehComp(' +str(i)+ ')',2)

VehInput = Vissim.Net.VehicleInputs.AddVehicleInput(0, Vissim.Net.Links.ItemByKey(67))
VehInput.SetAttValue('Name','Uber Initial Reserved Volume')
for i in range(1,9):
    VehInput.SetAttValue('Volume(' +str(i)+ ')', initial_volume)
    VehInput.SetAttValue('VehComp(' +str(i)+ ')',2)


VehInput = Vissim.Net.VehicleInputs.AddVehicleInput(0, Vissim.Net.Links.ItemByKey(10439))
VehInput.SetAttValue('Name','Airport taxi Initial Reserved Volume')
for i in range(1,9):
    VehInput.SetAttValue('Volume(' +str(i)+ ')', initial_volume)
    VehInput.SetAttValue('VehComp(' +str(i)+ ')',2)

VehInput = Vissim.Net.VehicleInputs.AddVehicleInput(0, Vissim.Net.Links.ItemByKey(71))
VehInput.SetAttValue('Name','Bmtc Initial Reserved Volume')
for i in range(1,9):
    VehInput.SetAttValue('Volume(' +str(i)+ ')', initial_volume)
#     VehInput.SetAttValue('VehComp(' +str(i)+ ')',3)

VehInput = Vissim.Net.VehicleInputs.AddVehicleInput(0, Vissim.Net.Links.ItemByKey(38))
VehInput.SetAttValue('Name','Private parking Initial Reserved Volume')
for i in range(1,9):
    VehInput.SetAttValue('Volume(' +str(i)+ ')', initial_volume)
    VehInput.SetAttValue('VehComp(' +str(i)+ ')',2)

VehInput = Vissim.Net.VehicleInputs.AddVehicleInput(0, Vissim.Net.Links.ItemByKey(10124))
VehInput.SetAttValue('Name','Private parking 1 Initial Reserved Volume')
for i in range(1,9):
    VehInput.SetAttValue('Volume(' +str(i)+ ')', initial_volume)
    VehInput.SetAttValue('VehComp(' +str(i)+ ')',2)

VehInput = Vissim.Net.VehicleInputs.AddVehicleInput(0, Vissim.Net.Links.ItemByKey(10289))
VehInput.SetAttValue('Name','Blusmart Initial Reserved Volume')
for i in range(1,9):
    VehInput.SetAttValue('Volume(' +str(i)+ ')', initial_volume_BLU)

### Initial volume assignment at pickup zone

In [125]:
initial_volume_1 = 100

In [126]:


VehInput = Vissim.Net.VehicleInputs.AddVehicleInput(0, Vissim.Net.Links.ItemByKey(10049))
VehInput.SetAttValue('Name','Uber Initial Volume')
for i in range(1,9):
    VehInput.SetAttValue('Volume(' +str(i)+ ')', initial_volume_1)
    VehInput.SetAttValue('VehComp(' +str(i)+ ')',2)


VehInput = Vissim.Net.VehicleInputs.AddVehicleInput(0, Vissim.Net.Links.ItemByKey(35))
VehInput.SetAttValue('Name','Blusmart Initial Volume')
for i in range(1,9):
    VehInput.SetAttValue('Volume(' +str(i)+ ')', initial_volume_1)
    VehInput.SetAttValue('VehComp(' +str(i)+ ')',2)


VehInput = Vissim.Net.VehicleInputs.AddVehicleInput(0, Vissim.Net.Links.ItemByKey(54))
VehInput.SetAttValue('Name','Airport taxi Initial Volume')
for i in range(1,9):
    VehInput.SetAttValue('Volume(' +str(i)+ ')', initial_volume_1)
    VehInput.SetAttValue('VehComp(' +str(i)+ ')',2)

VehInput = Vissim.Net.VehicleInputs.AddVehicleInput(0, Vissim.Net.Links.ItemByKey(5))
VehInput.SetAttValue('Name','Ola Initial Volume')
for i in range(1,9):
    VehInput.SetAttValue('Volume(' +str(i)+ ')', initial_volume_1)
    VehInput.SetAttValue('VehComp(' +str(i)+ ')',2)




In [94]:
VehRoutDesPark = Vissim.Net.VehicleRoutingDecisionsParking
for decision in VehRoutDesPark:
    decision.SetAttValue('ParkDur(1)',201)

In [ ]:
for decision in VehRoutDesPark:
    decision.SetAttValue('ParkDur(1)',201)

# Simulation initialization settings

In [40]:
Random_Seed = 42
Vissim.Simulation.SetAttValue('RandSeed', Random_Seed)

# Lot numbers
- ola_reserves_1 = 13
- ola_reserves_2 = 14
- ola_reserves_3 = 15
- ola_pickup = 16

In [42]:
Name = []
No = []
NumVehsEnter = []
NumVehsLeave = []
Occupancy = []
set_pedestrian_prev = set()
deque_pedestrian_curr = deque()

In [13]:
group_parking_lot_attributes = Vissim.Net.ParkLotGrps.GetMultipleAttributes(( 'Name','No','NumVehsEnter(Current,Total,All)','NumVehsEnter(Current,Last,All)','NumVehsPark(Current,Last,All)','NumVehsLeave(Current,Total,All)','OccupDurTot(Current,Last,All)','OccupRate(Current,Last,All)'))


In [67]:
keys_stage_area_park_dec = [94, 95]
for key in keys_stage_area_park_dec:
            decision = VehRoutDesPark.ItemByKey(key)
            decision.SetAttValue('ParkDur(1)',1)
            decision.SetAttValue('FullOccupBehav', 'DriveOn')

In [162]:
Vissim.Simulation.SetAttValue('UseMaxSimSpeed', True)
# Vissim.Simulation.RunSingleStep()
sim_res = Vissim.Simulation.AttValue('SimRes')
sim_len = Vissim.Simulation.AttValue('SimPeriod')



In [ ]:
for i in range(sim_res*sim_len):
    Vissim.Simulation.RunSingleStep()
    Vissim.SuspendUpdateGUI()
    print(i)
    print(Vissim.Simulation.AttValue('SimSec'))
    if i%10 == 0:
        Vissim.ResumeUpdateGUI(True) 

    if i == 1000:
        break

# Simulation for initialization

In [ ]:
keys = {950:[94,95] , 949:[97], 2527:[89,90,91], 2523:[96]}
ParkingLots = Vissim.Net.ParkingLots
VehRoutDesPark = Vissim.Net.VehicleRoutingDecisionsParking

for i in range(sim_res*sim_len):
    Vissim.Simulation.RunSingleStep()
    Vissim.SuspendUpdateGUI()
    # print(i)
    
    if i%10 == 0:
        Vissim.ResumeUpdateGUI(True) 

    if i == 6000:
        break
    if i%100 == 0:
        print(Vissim.Simulation.AttValue('SimSec'))
        
        for key,values in keys.items():
            Avail = ParkingLots.ItemByKey(key).AttValue('CurAvail')
            print(f'Number of free spots in the parking lot {ParkingLots.ItemByKey(key).AttValue("Name")} is {Avail}')
            if Avail > 1:
                for value in values:
                    decision = VehRoutDesPark.ItemByKey(value)
                    decision.SetAttValue('ParkDur(1)',1)
                    # decision.SetAttValue('FullOccupBehav', 'DriveOn')
                    # print(f'Parking lot {key} is available')
            elif Avail == 0 or Avail == 1:
                for value in values:
                    decision = VehRoutDesPark.ItemByKey(value)
                    decision.SetAttValue('ParkDur(1)',201)
                    # decision.SetAttValue('FullOccupBehav', 'Wait')
                    print(f'Parking lot {key} is full')

    
    

In [ ]:
keys = {950:[94,95] , 949:[97], 2527:[89,90,91], 2523:[96]}
# ParkingLots = Vissim.Net.ParkingLots
# VehRoutDesPark = Vissim.Net.VehicleRoutingDecisionsParking
deque_pedestrian_curr = deque()
deque_vehicle_curr = deque()

for i in range(sim_res*sim_len):
    ParkingLots = Vissim.Net.ParkingLots
    VehRoutDesPark = Vissim.Net.VehicleRoutingDecisionsParking
    Vissim.Simulation.RunSingleStep()
    # Vissim.SuspendUpdateGUI()
    pedestrian = Vissim.Net.Pedestrians
    count = 0

    for pedes in pedestrian:
            if pedes.AttValue('StaRoutNo') == 5 and (pedes.AttValue("MotionState") == 'WAITINGINQUEUE' or pedes.AttValue("MotionState") == 'WAITINGATQUEUEHEAD'):
                # print(1)
                count += 1
    # print(i)
    
    # if i%10 == 0:
    #     Vissim.ResumeUpdateGUI(True) 

    if i == 6000:
        break

    # if i > 2000:
    if i%100 == 0:
        # print(2)
        noPedestrian = False
        while count > 0:
            # print(count)

            """ Check if parking lot 950 is empty and if it is empty, then park the vehicles in the parking lot 950 """

            if ParkingLots.ItemByKey(950).AttValue('CurAvail') == ParkingLots.ItemByKey(950).AttValue('Capacity'):
                print("Parking lot 950 is empty")
                while ParkingLots.ItemByKey(950).AttValue('CurAvail') != 0:
                    for value in keys[950]:
                        decision = VehRoutDesPark.ItemByKey(value)
                        decision.SetAttValue('ParkDur(1)',1)
                    Vissim.Simulation.RunSingleStep()
                    i += 1
                    # count -= 1
                    continue      
                


            vehicles = Vissim.Net.Vehicles
            pedestrian = Vissim.Net.Pedestrians
            for pedes in pedestrian:
                if pedes.AttValue('StaRoutNo') == 5 and pedes.AttValue('PosInQueue') == 1 and pedes.AttValue("MotionState") == 'WAITINGATQUEUEHEAD':
                    pedes.SetAttValue('DwellTm', 0)
                    print(pedes.AttValue('No'))
                    Vissim.Simulation.RunSingleStep()
                    i += 1
                    break
            else:
                noPedestrian = True
              
            parkDur = 0
            veh_no = 0
            for veh in vehicles:
                if veh.AttValue('RoutDecNo') == 26 or veh.AttValue('RoutDecNo') == 27:
                    # print(veh.AttValue('ParkDurCur'))
                    # try:
                    if veh.AttValue('ParkDurCur') != None and veh.AttValue('ParkDurCur') > parkDur:
                        parkDur = veh.AttValue('ParkDurCur')
                        veh_no = veh.AttValue('No')
                        print(parkDur, veh_no)
                    # except:
                    #     # pass
                    #     print(veh.AttValue('ParkDurCur'))
                    # if veh.AttValue('ParkDurCur') > parkDur:
                    #     parkDur = veh.AttValue('ParkDurCur')
                    #     veh_no = veh.AttValue('No')
                    # print(parkDur, veh_no)
            if parkDur > 0 :
                # vehicles.ItemByKey(veh_no).SetAttValue('ParkDur', 0)
                vehicles.ItemByKey(veh_no).SetAttValue('DwellTm', 0)
                Vissim.Simulation.RunSingleStep()
                i += 1
            # Vissim.Simulation.RunSingleStep()
            print("time",Vissim.Simulation.AttValue('SimSec'))
            print("count",count)
            

            for j in range(1,21):
                # print(j)
                Vissim.Simulation.RunSingleStep()
                i += 1
            count -= 1

            
            # for key,values in keys.items():
            #     # Avail = ParkingLots.ItemByKey(key).AttValue('CurAvail')
            #     # print(f'Number of free spots in the parking lot {ParkingLots.ItemByKey(key).AttValue("Name")} is {Avail}')
            #     if key == 950:


        
            

# Initialization code

In [806]:
init_dwell_zero = [144,96,97,98,99,100,101,102,103,133,134,135,136,137,203,204,205,206,207,208,209,111,112,113,114,115,116,121,122,123,124,125,126,127,128,129,130]
init_infinite_dwell = [85,86,120,131,132,138,143,145,94,95,200,201,202,210,117,110]
VehRoutDesPark = Vissim.Net.VehicleRoutingDecisionsParking
for parklot in init_dwell_zero:
    # print(parklot)
    VehRoutDesPark.ItemByKey(parklot).SetAttValue('ParkDur(1)',201)

for parklot in init_infinite_dwell:
    VehRoutDesPark.ItemByKey(parklot).SetAttValue('ParkDur(1)',201)

In [ ]:
VehRoutDesPark = Vissim.Net.VehicleRoutingDecisionsParking

for lot in VehRoutDesPark:
    VehRoutDesPark.ItemByKey(lot).SetAttValue('ParkDur(1)',201)

In [807]:
def get_vehicle_info(vehicles):
    vehicle_info = {}
    for veh in vehicles:
        route_dec_no = veh.AttValue('RoutDecNo')
        park_state = veh.AttValue('ParkState')
        if park_state == 'ISPARKED':
            no = veh.AttValue('No')
            park_dur = veh.AttValue('ParkDurCur')
            vehicle_info[route_dec_no] = {'no': no, 'park_dur': park_dur}
    return vehicle_info

def set_dwell_time(vehicles, route_dec_no, dwell_time):
    for veh in vehicles:
        if veh.AttValue('RoutDecNo') == route_dec_no:
            vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', dwell_time)

# Simulation for initialization - I

In [808]:
vehicle_info = {85: {'no': 0, 'park_dur': 0},
                94: {'no': 0, 'park_dur': 0},
                95: {'no': 0, 'park_dur': 0},
                120: {'no': 0, 'park_dur': 0},
                132: {'no': 0, 'park_dur': 0},
                143: {'no': 0, 'park_dur': 0},
                200: {'no': 0, 'park_dur': 0},
                201: {'no': 0, 'park_dur': 0},
                202: {'no': 0, 'park_dur': 0}
                }

relevant_routes = [85, 120, 132, 143]
relevant_routes_2 = [200,201,202]
relevant_routes_3 = [94,95]
state = {85:False, 120:False, 132:False, 143:False}
state_2 = {200:False, 201:False, 202:False}
state_3 = {94:False, 95:False}

In [809]:
def vehicle_decisions(vehicle_info):
    vehicles = Vissim.Net.Vehicles

    for veh in vehicles:
        rout_dec_no = veh.AttValue('RoutDecNo')
        park_state = veh.AttValue('ParkState')
        # print(rout_dec_no)
        if rout_dec_no in vehicle_info and park_state == 'ISPARKED':
            # print(rout_dec_no)
            vehicle_info[rout_dec_no]['no'] = veh.AttValue('No')
            vehicle_info[rout_dec_no]['park_dur'] = veh.AttValue('ParkDurCur')
    return vehicle_info




In [810]:

for i in range(2000):
    vehicles = Vissim.Net.Vehicles
    Vissim.Simulation.RunSingleStep()
    
    if i % 50 == 0:
        # vehicle_info = get_vehicle_info(vehicles)
        vehicle_info = vehicle_decisions(vehicle_info)
        for route in relevant_routes:
            if vehicle_info[route]['park_dur'] != 0:
                state[route] = True
                # set_dwell_time(vehicles, route, 0)
        for route in relevant_routes_2:
            if vehicle_info[route]['park_dur'] != 0:
                state_2[route] = True
                # set_dwell_time(vehicles, route, 0)
        for route in relevant_routes_3:
            if vehicle_info[route]['park_dur'] != 0:
                state_3[route] = True
                # set_dwell_time(vehicles, route, 0)
    # if all values in state are True and any value in state_2 is True and any value in state_3 is True then set dwell time to 0 for vehicles in relevant_routes and one of the vehicles in relevant_routes_2 and relevant_routes_3 with the highest dwell time
    if all(state.values()) and any(state_2.values()) and any(state_3.values()):
        for route in relevant_routes:
            if state[route]:
                set_dwell_time(vehicles, route, 0)
        for route in relevant_routes_2:
            if state_2[route]:
                set_dwell_time(vehicles, route, 0)
                break
        for route in relevant_routes_3:
            if state_3[route]:
                set_dwell_time(vehicles, route, 0)
                break

        for j in range(1,21):
            Vissim.Simulation.RunSingleStep()
            i += 1

        break


In [651]:
Vissim.Net.ParkingLots.ItemByKey(2589).AttValue('CurAvail')

1

# Initialization II

In [815]:
start = i
end = 6000

In [814]:
VehRoutDesPark = Vissim.Net.VehicleRoutingDecisionsParking

# Define the routing decisions and visited dictionaries
routing_decisions = {104: [103],103: [102],102: [101],101: [100],100: [99],99: [98],98: [97, 96]}
park_lot =[94,95]

routing_decisions_2 = {145: [144]}
park_lot_2 = [143]

routing_decisions_3 = {138: [137], 137: [136], 136: [135], 135: [134], 134: [133]}
park_lot_3 = [132]

routing_decisions_4 = {210:[209], 209: [208], 208: [207], 207: [206], 206: [205], 205: [204], 204: [203]}
park_lot_4 = [202,201,200]

routing_decisions_5 = {131: [130], 130: [129], 129: [128], 128: [127], 127: [126], 126: [125], 125: [124],124: [123], 123: [122], 122: [121]}
park_lot_5 = [120]

visited = {key: False for key in routing_decisions.keys()}
visited_2 = {key: False for key in routing_decisions_2.keys()}
visited_3 = {key: False for key in routing_decisions_3.keys()}
visited_4 = {key: False for key in routing_decisions_4.keys()}
visited_5 = {key: False for key in routing_decisions_5.keys()}

In [812]:
def staging_veh(vissim):
    vehicle_info = {85: {'no': 0, 'park_dur': 0},
                94: {'no': 0, 'park_dur': 0},
                95: {'no': 0, 'park_dur': 0},
                120: {'no': 0, 'park_dur': 0},
                132: {'no': 0, 'park_dur': 0},
                143: {'no': 0, 'park_dur': 0},
                200: {'no': 0, 'park_dur': 0},
                201: {'no': 0, 'park_dur': 0},
                202: {'no': 0, 'park_dur': 0}
                }

    for veh in vissim.Net.Vehicles:
        rout_dec_no = veh.AttValue('RoutDecNo')
        park_state = veh.AttValue('ParkState')

        if rout_dec_no in vehicle_info and park_state == 'ISPARKED':
            vehicle_info[rout_dec_no]['no'] = veh.AttValue('No')
            vehicle_info[rout_dec_no]['park_dur'] = veh.AttValue('ParkDurCur')
    return vehicle_info


In [813]:
def process_parking_lot(veh, VehRoutDesPark, vehicle_info, current_dec, next_dec_list, visited, park_lot):
    
    if veh.AttValue('RoutDecNo') == current_dec and veh.AttValue('ParkState') == 'ISPARKED' and not visited[current_dec]:
        best_vehicle = max(park_lot, key=lambda x: vehicle_info[x]['park_dur'])
        print(best_vehicle, vehicle_info[best_vehicle]['no'])
        if vehicle_info[best_vehicle]['no'] > 0:

            print(veh.AttValue('RoutDecNo'), veh.AttValue('ParkState'))
            for next_dec in next_dec_list:
                VehRoutDesPark.ItemByKey(next_dec).SetAttValue('ParkDur(1)', 201)
            
            vehicles.ItemByKey(vehicle_info[best_vehicle]['no']).SetAttValue('DwellTm', 0)
            visited[current_dec] = True 
            # Vissim.Simulation.RunSingleStep()
            

In [920]:
Vissim.Net.ParkingLots.ItemByKey(2590).AttValue(r'Last:VehParking\No')

8

In [1016]:
Vissim.Net.ParkingLots

<win32com.gen_py.Vissim Object Library 24.0 64 Bit.IParkingLotContainer instance at 0x1592335694352>

# Init Part

In [1015]:
uber_go_lot_dec = [94,95,96,97,98,99,100,101,102,103]
uber_go_stage = [94,95]
uber_go_lot = [i for i in range(2581,2591)]

uber_xl_lot_dec = [143,144]
uber_xl_stage = [143]
uber_xl_lot = [i for i in range(5998,6001)]

blusmart_lot_dec = [132,133,134,135,136,137]
blusmart_stage = [132]
blusmart_lot = [i for i in range(9000,9007)]

ola_mini_lot_dec = [120,121,122,123,124,125,126,127,128,129,130]
ola_mini_stage = [120]
ola_mini_lot = [i for i in range(5871,5883)]

ola_suv_lot_dec = [85]
ola_suv_stage = [85]
ola_suv_lot = [i for i in range(6999,7001)]

airport_taxi_lot_dec = [198,199,200,201,202,203,204,205,206,207,208,209]
airport_taxi_stage = [200,199,198]
airport_taxi_lot = [i for i in range(7997,8008)]

assigned = [-1 for i in range(10000)]

In [1017]:
def find_park_duration(vehicle_list):

    vehicle_info = {85: {'no': 0, 'park_dur': 0},
                94: {'no': 0, 'park_dur': 0},
                95: {'no': 0, 'park_dur': 0},
                120: {'no': 0, 'park_dur': 0},
                132: {'no': 0, 'park_dur': 0},
                143: {'no': 0, 'park_dur': 0},
                200: {'no': 0, 'park_dur': 0},
                199: {'no': 0, 'park_dur': 0},
                198: {'no': 0, 'park_dur': 0}
                }
    # print(assigned[2583:2591])
    
    for veh in vehicle_list:


        for dec_no  in uber_go_lot_dec:
            # print(dec_no)
            # continue
            if veh.AttValue('DestParkLot') == None:
                continue
            next_dest = 2583 if veh.AttValue('RoutDecNo') in uber_go_stage else int(veh.AttValue('DestParkLot')) + 1

            if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] == -1 or assigned[next_dest] == None):
                if (veh.AttValue('RoutDecNo') == 94 or veh.AttValue('RoutDecNo') == 95):
                    # print(veh.AttValue('RoutDecNo'), veh.AttValue('ParkDurCur'))
                    vehicle_info[veh.AttValue('RoutDecNo')]['no'] = veh.AttValue('No')
                    vehicle_info[veh.AttValue('RoutDecNo')]['park_dur'] = veh.AttValue('ParkDurCur')
                    continue
                assigned[next_dest] = veh.AttValue('No')
                Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)
            




        for dec_no in uber_xl_lot_dec:
            if veh.AttValue('DestParkLot') == None:
                continue
            next_dest = int(veh.AttValue('DestParkLot')) + 1
            if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] == -1 or assigned[next_dest] == None):
                assigned[next_dest] = veh.AttValue('No')
                Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)





        for dec_no in blusmart_lot_dec:
            if veh.AttValue('DestParkLot') == None:
                continue
            next_dest = int(veh.AttValue('DestParkLot')) + 1
            if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] == -1 or assigned[next_dest] == None):
                assigned[next_dest] = veh.AttValue('No')
                Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)




        for dec_no in airport_taxi_lot_dec:
            if veh.AttValue('DestParkLot') == None:
                continue
            next_dest = 8000 if veh.AttValue('RoutDecNo') in airport_taxi_stage else int(veh.AttValue('DestParkLot')) + 1
            if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] == -1 or assigned[next_dest] == None):
                if (veh.AttValue('RoutDecNo') == 200 or veh.AttValue('RoutDecNo') == 199 or veh.AttValue('RoutDecNo') == 198):
                    # print(veh.AttValue('RoutDecNo'), veh.AttValue('ParkDurCur'))
                    vehicle_info[veh.AttValue('RoutDecNo')]['no'] = veh.AttValue('No')
                    vehicle_info[veh.AttValue('RoutDecNo')]['park_dur'] = veh.AttValue('ParkDurCur')
                    continue
                assigned[next_dest] = veh.AttValue('No')
                Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)


                

        for dec_no in ola_mini_lot_dec:
            if veh.AttValue('DestParkLot') == None:
                continue
            next_dest = int(veh.AttValue('DestParkLot')) + 1
            # print(veh.AttValue('RoutDecNo'), veh.AttValue('ParkState'), next_dest)
            if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] == -1 or assigned[next_dest] == None):
                # print(veh.AttValue('RoutDecNo'), veh.AttValue('ParkState'), next_dest)
                assigned[next_dest] = veh.AttValue('No')
                Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)

        for dec_no in ola_suv_lot_dec:
            if veh.AttValue('DestParkLot') == None:
                continue
            next_dest = int(veh.AttValue('DestParkLot')) + 1
            if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] == -1 or assigned[next_dest] == None):
                assigned[next_dest] = veh.AttValue('No')
                Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)

             
        
    return vehicle_info

In [ ]:
vehicle_info

In [1018]:

# Main simulation loop
for i in range(6001):
    vehicles = Vissim.Net.Vehicles
    Vissim.Simulation.RunSingleStep()
    VehRoutDesPark = Vissim.Net.VehicleRoutingDecisionsParking
    
    # Run the following code every 100 simulation steps
    if i % 50 == 0:
        print(Vissim.Simulation.AttValue('SimSec'))
        vehicle_info = {85: {'no': 0, 'park_dur': 0},
                94: {'no': 0, 'park_dur': 0},
                95: {'no': 0, 'park_dur': 0},
                120: {'no': 0, 'park_dur': 0},
                132: {'no': 0, 'park_dur': 0},
                143: {'no': 0, 'park_dur': 0},
                200: {'no': 0, 'park_dur': 0},
                199: {'no': 0, 'park_dur': 0},
                198: {'no': 0, 'park_dur': 0}
                }
        vehicle_info =  find_park_duration(vehicles)

        if vehicle_info[94]['park_dur'] > vehicle_info[95]['park_dur']:
            Vissim.Net.Vehicles.ItemByKey(vehicle_info[94]['no']).SetAttValue('DwellTm', 0)
            assigned[2583] = vehicle_info[94]['no']
        elif vehicle_info[94]['park_dur'] < vehicle_info[95]['park_dur']:
            Vissim.Net.Vehicles.ItemByKey(vehicle_info[95]['no']).SetAttValue('DwellTm', 0)
            assigned[2583] = vehicle_info[95]['no']



        if vehicle_info[200]['park_dur'] >= vehicle_info[199]['park_dur'] and vehicle_info[200]['park_dur'] >= vehicle_info[198]['park_dur'] and vehicle_info[200]['park_dur'] > 0:
            Vissim.Net.Vehicles.ItemByKey(vehicle_info[200]['no']).SetAttValue('DwellTm', 0)
            assigned[8000] = vehicle_info[200]['no']
        elif vehicle_info[199]['park_dur'] >= vehicle_info[200]['park_dur'] and vehicle_info[199]['park_dur'] >= vehicle_info[198]['park_dur'] and vehicle_info[199]['park_dur'] > 0:
            Vissim.Net.Vehicles.ItemByKey(vehicle_info[199]['no']).SetAttValue('DwellTm', 0)
            assigned[8000] = vehicle_info[199]['no']
        elif vehicle_info[198]['park_dur'] > vehicle_info[200]['park_dur'] and vehicle_info[198]['park_dur'] > vehicle_info[199]['park_dur']:
            Vissim.Net.Vehicles.ItemByKey(vehicle_info[198]['no']).SetAttValue('DwellTm', 0)
            assigned[8000] = vehicle_info[198]['no']

        for lot_no in uber_go_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            # if lot_no == 2583:
            #     print(assigned[lot_no], lot.AttValue('CurAvail'), lot.AttValue(r'Last:VehParking\No'), vehicle_list.ItemByKey(assigned[lot_no]).AttValue('DestParkLot') if assigned[lot_no] != -1 else -1)
            if lot.AttValue('CurAvail') == 0:
                # print(lot.AttValue('No'), "is full")
                assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
            if assigned[lot_no] != -1 and assigned[lot_no] != None:
                # print(assigned[lot_no], vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'))
                # print(assigned[lot_no])
                if lot_no != vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'):
                    assigned[lot.AttValue('No')] = -1


        for lot_no in airport_taxi_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            if lot.AttValue('CurAvail') == 0:
                print(lot.AttValue('No'), "is full")
                assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
            if assigned[lot_no] != -1 and assigned[lot_no] != None:
                print(assigned[lot_no], vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'))
                # print(assigned[lot_no])
                if lot_no != vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'):
                    assigned[lot.AttValue('No')] = -1

        for lot_no in ola_mini_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            if lot.AttValue('CurAvail') == 0:
                # print(lot.AttValue('No'), "is full")
                assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
            if assigned[lot_no] != -1 and assigned[lot_no] != None:
                # print(assigned[lot_no], vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'))
                # print(assigned[lot_no])
                if lot_no != vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'):
                    assigned[lot.AttValue('No')] = -1

        for lot_no in uber_xl_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            if lot.AttValue('CurAvail') == 0:
                # print(lot.AttValue('No'), "is full")
                assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
            if assigned[lot_no] != -1 and assigned[lot_no] != None:
                # print(assigned[lot_no], vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'))
                # print(assigned[lot_no])
                if lot_no != vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'):
                    assigned[lot.AttValue('No')] = -1

        for lot_no in blusmart_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            if lot.AttValue('CurAvail') == 0:
                # print(lot.AttValue('No'), "is full")
                assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
            if assigned[lot_no] != -1 and assigned[lot_no] != None:
                # print(assigned[lot_no], vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'))
                # print(assigned[lot_no])
                if lot_no != vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'):
                    assigned[lot.AttValue('No')] = -1

        for lot_no in ola_suv_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            if lot.AttValue('CurAvail') == 0:
                # print(lot.AttValue('No'), "is full")
                assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
            if assigned[lot_no] != -1 and assigned[lot_no] != None:
                # print(assigned[lot_no], vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'))
                # print(assigned[lot_no])
                if lot_no != vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'):
                    assigned[lot.AttValue('No')] = -1

        

        

0.1
5.1
10.1
15.1
20.1
25.1
30.1
35.1
40.1
45.1
50.1
55.1
60.1
65.1
70.1
75.1
80.1
85.1
90.1
95.1
7998 is full
100.1
199 4.100000000000001
7998 is full
52 7998
105.1
8000 is full
110.1
8000 is full
115.1
8000 is full
120.1
8000 is full
125.1
7998 is full
70 7998
8000 is full
52 8000
130.1
199 6.199999999999994
7998 is full
70 7998
8001 is full
135.1
8000 is full
8001 is full
52 8001
140.1
8000 is full
8002 is full
145.1
8000 is full
8002 is full
52 8002
150.1
8000 is full
8003 is full
155.1
8000 is full
70 8000
8003 is full
52 8003
160.1
8001 is full
8004 is full
165.1
8001 is full
70 8001
8004 is full
52 8004
170.1
8002 is full
8005 is full
175.1
8002 is full
70 8002
8005 is full
52 8005
180.1
8003 is full
8006 is full
185.1
7997 is full
8003 is full
70 8003
8006 is full
52 8006
190.1
198 2.3000000000000007
7997 is full
116 7997
8004 is full
8007 is full
195.1
7997 is full
8004 is full
70 8004
8007 is full
52 8007
200.1
198 3.0000000000000013
7997 is full
120 7997
8005 is full
8007 is

# Main Simulation

In [1019]:
# pedes_area = [51,52,50,22,29,37]
veh_lot_pedes_area = {2590:51, 6000:52, 9006:50, 5882:22, 7000:29, 8007:37}
count_veh = {2590:0, 6000:0, 9006:0, 5882:0, 7000:0, 8007:0}
prev_passenger_in_queue = {51:0, 52:0, 50:0, 22:0, 29:0, 37:0}
reserved_vehs = {2590:{'no' : 0, 'park_dur': 0}, 6000:{'no' : 0, 'park_dur': 0}, 9006:{'no' : 0, 'park_dur': 0}, 5882:{'no' : 0, 'park_dur': 0}, 7000:{'no' : 0, 'park_dur': 0}, 8007:{'no' : 0, 'park_dur': 0}}
# prev_passenger_in_queue = 0

In [1027]:
def find_park_duration(vehicle_list, reserved_vehs):

    vehicle_info = {85: {'no': 0, 'park_dur': 0},
                94: {'no': 0, 'park_dur': 0},
                95: {'no': 0, 'park_dur': 0},
                120: {'no': 0, 'park_dur': 0},
                132: {'no': 0, 'park_dur': 0},
                143: {'no': 0, 'park_dur': 0},
                200: {'no': 0, 'park_dur': 0},
                199: {'no': 0, 'park_dur': 0},
                198: {'no': 0, 'park_dur': 0}
                }

    # print(assigned[2583:2591])
    
    for veh in vehicle_list:
        if veh.AttValue('DestParkLot') == None:
                continue

# uber go
        if  veh.AttValue('RoutDecNo') != None and (veh.AttValue('RoutDecNo')>= 38 and veh.AttValue('RoutDecNo') <= 46) and veh.AttValue('ParkState') == 'ISPARKED':
                    if veh.AttValue('ParkDurCur') > reserved_vehs[2590]['park_dur']:
                        reserved_vehs[2590]['park_dur'] = veh.AttValue('ParkDurCur')
                        reserved_vehs[2590]['no'] = veh.AttValue('No')
# uber xl
        if  veh.AttValue('RoutDecNo') != None and (veh.AttValue('RoutDecNo') == 37) and veh.AttValue('ParkState') == 'ISPARKED':
                    if veh.AttValue('ParkDurCur') > reserved_vehs[6000]['park_dur']:
                        reserved_vehs[6000]['park_dur'] = veh.AttValue('ParkDurCur')
                        reserved_vehs[6000]['no'] = veh.AttValue('No')
# blusmart
        if  veh.AttValue('RoutDecNo') != None and (veh.AttValue('RoutDecNo') == 36) and veh.AttValue('ParkState') == 'ISPARKED':
                    if veh.AttValue('ParkDurCur') > reserved_vehs[9006]['park_dur']:
                        reserved_vehs[9006]['park_dur'] = veh.AttValue('ParkDurCur')
                        reserved_vehs[9006]['no'] = veh.AttValue('No')
# ola mini
        if  veh.AttValue('RoutDecNo') != None and (veh.AttValue('RoutDecNo')>= 29 and veh.AttValue('RoutDecNo') <= 34) and veh.AttValue('ParkState') == 'ISPARKED':
                    if veh.AttValue('ParkDurCur') > reserved_vehs[5882]['park_dur']:
                        reserved_vehs[5882]['park_dur'] = veh.AttValue('ParkDurCur')
                        reserved_vehs[5882]['no'] = veh.AttValue('No')

# ola suv
        if  veh.AttValue('RoutDecNo') != None and (veh.AttValue('RoutDecNo') == 35) and veh.AttValue('ParkState') == 'ISPARKED':
                    if veh.AttValue('ParkDurCur') > reserved_vehs[7000]['park_dur']:
                        reserved_vehs[7000]['park_dur'] = veh.AttValue('ParkDurCur')
                        reserved_vehs[7000]['no'] = veh.AttValue('No')
# airport taxi
        if  veh.AttValue('RoutDecNo') != None and (veh.AttValue('RoutDecNo')>= 58 and veh.AttValue('RoutDecNo') <= 84) and veh.AttValue('ParkState') == 'ISPARKED':
                    if veh.AttValue('ParkDurCur') > reserved_vehs[8007]['park_dur']:
                        reserved_vehs[8007]['park_dur'] = veh.AttValue('ParkDurCur')
                        reserved_vehs[8007]['no'] = veh.AttValue('No')




        for dec_no  in uber_go_lot_dec:
            # print(dec_no)
            # continue
            # if veh.AttValue('DestParkLot') == None:
            #     continue
            next_dest = 2583 if veh.AttValue('RoutDecNo') in uber_go_stage else int(veh.AttValue('DestParkLot')) + 1

            if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] == -1 or assigned[next_dest] == None):
                if (veh.AttValue('RoutDecNo') == 94 or veh.AttValue('RoutDecNo') == 95):
                    # print(veh.AttValue('RoutDecNo'), veh.AttValue('ParkDurCur'))
                    vehicle_info[veh.AttValue('RoutDecNo')]['no'] = veh.AttValue('No')
                    vehicle_info[veh.AttValue('RoutDecNo')]['park_dur'] = veh.AttValue('ParkDurCur')
                    continue
                assigned[next_dest] = veh.AttValue('No')
                Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)
            




        for dec_no in uber_xl_lot_dec:
            # if veh.AttValue('DestParkLot') == None:
            #     continue
            next_dest = int(veh.AttValue('DestParkLot')) + 1
            if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] == -1 or assigned[next_dest] == None):
                assigned[next_dest] = veh.AttValue('No')
                Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)





        for dec_no in blusmart_lot_dec:
            # if veh.AttValue('DestParkLot') == None:
            #     continue
            next_dest = int(veh.AttValue('DestParkLot')) + 1
            if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] == -1 or assigned[next_dest] == None):
                assigned[next_dest] = veh.AttValue('No')
                Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)




        for dec_no in airport_taxi_lot_dec:
            # if veh.AttValue('DestParkLot') == None:
            #     continue
            next_dest = 8000 if veh.AttValue('RoutDecNo') in airport_taxi_stage else int(veh.AttValue('DestParkLot')) + 1
            if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] == -1 or assigned[next_dest] == None):
                if (veh.AttValue('RoutDecNo') == 200 or veh.AttValue('RoutDecNo') == 199 or veh.AttValue('RoutDecNo') == 198):
                    # print(veh.AttValue('RoutDecNo'), veh.AttValue('ParkDurCur'))
                    vehicle_info[veh.AttValue('RoutDecNo')]['no'] = veh.AttValue('No')
                    vehicle_info[veh.AttValue('RoutDecNo')]['park_dur'] = veh.AttValue('ParkDurCur')
                    continue
                assigned[next_dest] = veh.AttValue('No')
                Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)


                

        for dec_no in ola_mini_lot_dec:
            # if veh.AttValue('DestParkLot') == None:
            #     continue
            next_dest = int(veh.AttValue('DestParkLot')) + 1
            # print(veh.AttValue('RoutDecNo'), veh.AttValue('ParkState'), next_dest)
            if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] == -1 or assigned[next_dest] == None):
                # print(veh.AttValue('RoutDecNo'), veh.AttValue('ParkState'), next_dest)
                assigned[next_dest] = veh.AttValue('No')
                Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)

                

        for dec_no in ola_suv_lot_dec:
            # if veh.AttValue('DestParkLot') == None:
            #     continue
            next_dest = int(veh.AttValue('DestParkLot')) + 1
            if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1 and (assigned[next_dest] == -1 or assigned[next_dest] == None):
                assigned[next_dest] = veh.AttValue('No')
                Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)


    return vehicle_info, reserved_vehs

In [1028]:
for i in range(6001):
    vehicles = Vissim.Net.Vehicles
    Vissim.Simulation.RunSingleStep()
    VehRoutDesPark = Vissim.Net.VehicleRoutingDecisionsParking

    pedestrian = Vissim.Net.Pedestrians
    
    
    
    # Run the following code every 100 simulation steps
    if i % 50 == 0:
        print(Vissim.Simulation.AttValue('SimSec'))

        vehicle_info = {85: {'no': 0, 'park_dur': 0},
                94: {'no': 0, 'park_dur': 0},
                95: {'no': 0, 'park_dur': 0},
                120: {'no': 0, 'park_dur': 0},
                132: {'no': 0, 'park_dur': 0},
                143: {'no': 0, 'park_dur': 0},
                200: {'no': 0, 'park_dur': 0},
                199: {'no': 0, 'park_dur': 0},
                198: {'no': 0, 'park_dur': 0}
                }
        vehicle_info,reserved_vehs =  find_park_duration(vehicles,reserved_vehs)

        if vehicle_info[94]['park_dur'] > vehicle_info[95]['park_dur']:
            Vissim.Net.Vehicles.ItemByKey(vehicle_info[94]['no']).SetAttValue('DwellTm', 0)
            assigned[2583] = vehicle_info[94]['no']
        elif vehicle_info[94]['park_dur'] < vehicle_info[95]['park_dur']:
            Vissim.Net.Vehicles.ItemByKey(vehicle_info[95]['no']).SetAttValue('DwellTm', 0)
            assigned[2583] = vehicle_info[95]['no']

        if vehicle_info[200]['park_dur'] >= vehicle_info[199]['park_dur'] and vehicle_info[200]['park_dur'] >= vehicle_info[198]['park_dur'] and vehicle_info[200]['park_dur'] > 0:
            Vissim.Net.Vehicles.ItemByKey(vehicle_info[200]['no']).SetAttValue('DwellTm', 0)
            assigned[8000] = vehicle_info[200]['no']
        elif vehicle_info[199]['park_dur'] >= vehicle_info[200]['park_dur'] and vehicle_info[199]['park_dur'] >= vehicle_info[198]['park_dur'] and vehicle_info[199]['park_dur'] > 0:
            Vissim.Net.Vehicles.ItemByKey(vehicle_info[199]['no']).SetAttValue('DwellTm', 0)
            assigned[8000] = vehicle_info[199]['no']
        elif vehicle_info[198]['park_dur'] > vehicle_info[200]['park_dur'] and vehicle_info[198]['park_dur'] > vehicle_info[199]['park_dur']:
            Vissim.Net.Vehicles.ItemByKey(vehicle_info[198]['no']).SetAttValue('DwellTm', 0)
            assigned[8000] = vehicle_info[198]['no']


        for lot_no in uber_go_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            # if lot_no == 2583:
            #     print(assigned[lot_no], lot.AttValue('CurAvail'), lot.AttValue(r'Last:VehParking\No'), vehicle_list.ItemByKey(assigned[lot_no]).AttValue('DestParkLot') if assigned[lot_no] != -1 else -1)
            if lot.AttValue('CurAvail') == 0:
                # print(lot.AttValue('No'), "is full")
                assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
            if assigned[lot_no] != -1 and assigned[lot_no] != None:
                # print(assigned[lot_no], vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'))
                # print(assigned[lot_no])
                if lot_no != vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'):
                    assigned[lot.AttValue('No')] = -1

        for lot_no in airport_taxi_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            if lot.AttValue('CurAvail') == 0:
                # print(lot.AttValue('No'), "is full")
                assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
            if assigned[lot_no] != -1 and assigned[lot_no] != None:
                # print(assigned[lot_no], vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'))
                # print(assigned[lot_no])
                if lot_no != vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'):
                    assigned[lot.AttValue('No')] = -1

        for lot_no in ola_mini_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            if lot.AttValue('CurAvail') == 0:
                # print(lot.AttValue('No'), "is full")
                assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
            if assigned[lot_no] != -1 and assigned[lot_no] != None:
                # print(assigned[lot_no], vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'))
                # print(assigned[lot_no])
                if lot_no != vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'):
                    assigned[lot.AttValue('No')] = -1

        for lot_no in uber_xl_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            if lot.AttValue('CurAvail') == 0:
                # print(lot.AttValue('No'), "is full")
                assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
            if assigned[lot_no] != -1 and assigned[lot_no] != None:
                # print(assigned[lot_no], vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'))
                # print(assigned[lot_no])
                if lot_no != vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'):
                    assigned[lot.AttValue('No')] = -1

        for lot_no in blusmart_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            if lot.AttValue('CurAvail') == 0:
                # print(lot.AttValue('No'), "is full")
                assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
            if assigned[lot_no] != -1 and assigned[lot_no] != None:
                # print(assigned[lot_no], vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'))
                # print(assigned[lot_no])
                if lot_no != vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'):
                    assigned[lot.AttValue('No')] = -1

        for lot_no in ola_suv_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            if lot.AttValue('CurAvail') == 0:
                # print(lot.AttValue('No'), "is full")
                assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
            if assigned[lot_no] != -1 and assigned[lot_no] != None:
                # print(assigned[lot_no], vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'))
                # print(assigned[lot_no])
                if lot_no != vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'):
                    assigned[lot.AttValue('No')] = -1       



        # if passenger is at 1st position in the queue and vehicle is parked
                    
        for veh_lot, pedes_area in veh_lot_pedes_area.items():
            pedes_no = None
            pedes_pos = None
            

            for pedes in pedestrian:
                if pedes.AttValue('ConstrElNo') == pedes_area and pedes.AttValue('PosInQueue') == 1 and pedes.AttValue("MotionState") == 'WAITINGATQUEUEHEAD':
                    pedes_no = pedes.AttValue('No')
                    pedes_pos = pedes.AttValue('PosInQueue')
                    break

            veh_no = Vissim.Net.ParkingLots.ItemByKey(veh_lot).AttValue(r'Last:VehParking\No')

            if veh_no != None and pedes_no != None and pedes_pos == 1:
              
                Vissim.Net.Vehicles.ItemByKey(veh_no).SetAttValue('DwellTm', 0)
                
                Vissim.Net.Pedestrians.ItemByKey(pedes_no).SetAttValue('DwellTm', 0)

                # print(veh_no, pedes_no, pedes_pos, veh_lot, pedes_area, count_veh[veh_lot])

                count_veh[veh_lot] += 1
                prev_passenger_in_queue[pedes_area] = pedes_no

            elif veh_no == None and pedes_no != None and pedes_pos == 1:
                if prev_passenger_in_queue[pedes_area] != pedes_no:
                    count_veh[veh_lot] += 1

        for veh_lot, count in count_veh.items():
            # print(veh_lot, count)
            if count > 0:
                if reserved_vehs[veh_lot]['no'] != 0:
                    Vissim.Net.Vehicles.ItemByKey(reserved_vehs[veh_lot]['no']).SetAttValue('DwellTm', 0)
                    reserved_vehs[veh_lot]['no'] = 0
                    reserved_vehs[veh_lot]['park_dur'] = 0
                    count_veh[veh_lot] -= 1

    


3090.6999999999994
3095.6999999999994
3100.6999999999994
3105.6999999999994
3110.6999999999994
3115.6999999999994
3120.6999999999994
3125.6999999999994
3130.6999999999994
3135.6999999999994
3140.6999999999994
3145.6999999999994
3150.6999999999994


com_error: (-2147418111, 'Call was rejected by callee.', None, None)

In [991]:
for pedes in pedestrian:
            if pedes.AttValue('ConstrElNo') == 52 and pedes.AttValue('PosInQueue') == 1 and pedes.AttValue("MotionState") == 'WAITINGATQUEUEHEAD':
                print(pedes.AttValue('No'))

In [982]:
pedes = Vissim.Net.Pedestrians.ItemByKey(12).AttValue('PosInQueue')

In [983]:
print(pedes)

3


# End of Simulation

In [ ]:
for lot_no in ola_mini_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            print(lot)

In [ ]:

# Main simulation loop
for i in range(start, end):
    vehicles = Vissim.Net.Vehicles
    Vissim.Simulation.RunSingleStep()
    VehRoutDesPark = Vissim.Net.VehicleRoutingDecisionsParking
    
    # Run the following code every 100 simulation steps
    if i % 50 == 0:
        vehicle_info = staging_veh(Vissim)

        # veh_no, vehIn_94_ParkDur = 0, 0
        # vehIn_95, vehIn_95_ParkDur = 0, 0
        # veh_94 = Vissim.Net.ParkingLots.ItemByKey(2582).AttValue(r'Last:VehParking\No')
        # veh_95 = Vissim.Net.ParkingLots.ItemByKey(2581).AttValue(r'Last:VehParking\No')
        # if vehicles.ItemByKey(veh_94).AttValue('RoutDecNo') == 94 and vehicles.ItemByKey(veh_94).AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(2583).AttValue('CurAvail') == 1:
        #      veh_no, vehIn_94_ParkDur = veh.AttValue('No'), veh.AttValue('ParkDurCur')
        # if vehicles.ItemByKey(veh_95).AttValue('RoutDecNo') == 95 and vehicles.ItemByKey(veh_95).AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(2583).AttValue('CurAvail') == 1:
        #     vehIn_95, vehIn_95_ParkDur = veh.AttValue('No'), veh.AttValue('ParkDurCur')

        # for veh in vehicles:
        #     if veh.AttValue('RoutDecNo') in [94, 95]:
        #         continue
        #     if veh.AttValue('RoutDecNo') == 94 and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(2583).AttValue('CurAvail') == 1:
        #         veh_no, vehIn_94_ParkDur = veh.AttValue('No'), veh.AttValue('ParkDurCur')
        #     if veh.AttValue('RoutDecNo') == 95 and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(2583).AttValue('CurAvail') == 1:
        #         vehIn_95, vehIn_95_ParkDur = veh.AttValue('No'), veh.AttValue('ParkDurCur')



        # veh_no, vehIn_94_ParkDur = find_park_duration(vehicles, 94)
        # vehIn_95, vehIn_95_ParkDur = find_park_duration(vehicles, 95)
        # print(veh_no, vehIn_94_ParkDur, vehIn_95, vehIn_95_ParkDur)

        # if vehIn_94_ParkDur > vehIn_95_ParkDur:
        #     Vissim.Net.Vehicles.ItemByKey(veh_no).SetAttValue('DwellTm', 0)
        # elif vehIn_94_ParkDur < vehIn_95_ParkDur:
        #     Vissim.Net.Vehicles.ItemByKey(vehIn_95).SetAttValue('DwellTm', 0)

        # for i in  [94,95,96,97,98,99,100,101,102,103]:
            # if i in [96,97,98,99,100,101,102,103]:
            #     continue
            # print(assigned[2583:2591])
        vehicle_info =  find_park_duration(vehicles)

        if vehicle_info[94]['park_dur'] > vehicle_info[95]['park_dur']:
            Vissim.Net.Vehicles.ItemByKey(vehicle_info[94]['no']).SetAttValue('DwellTm', 0)
            assigned[2583] = vehicle_info[94]['no']
        elif vehicle_info[94]['park_dur'] < vehicle_info[95]['park_dur']:
            Vissim.Net.Vehicles.ItemByKey(vehicle_info[95]['no']).SetAttValue('DwellTm', 0)
            assigned[2583] = vehicle_info[95]['no']

        for lot_no in uber_go_lot:
            lot = Vissim.Net.ParkingLots.ItemByKey(lot_no)
            # if lot_no == 2583:
            #     print(assigned[lot_no], lot.AttValue('CurAvail'), lot.AttValue(r'Last:VehParking\No'), vehicle_list.ItemByKey(assigned[lot_no]).AttValue('DestParkLot') if assigned[lot_no] != -1 else -1)
            if lot.AttValue('CurAvail') == 0:
                print(lot.AttValue('No'), "is full")
                assigned[lot.AttValue('No')] = lot.AttValue(r'Last:VehParking\No')
            if assigned[lot_no] != -1 and assigned[lot_no] != None:
                print(assigned[lot_no], vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'))
                # print(assigned[lot_no])
                if lot_no != vehicles.ItemByKey(assigned[lot_no]).AttValue('DestParkLot'):
                    assigned[lot.AttValue('No')] = -1

        # for veh in vehicles:
        # # Loop through routing decisions and check if they are visited
            
        #     for current_dec, next_dec_list in routing_decisions.items():
        #         find_park_duration(vehicles, current_dec)
                # process_parking_lot(veh,VehRoutDesPark, vehicle_info, current_dec, next_dec_list, visited, park_lot)


            # for current_dec, next_dec_list in routing_decisions_2.items():
            #     process_parking_lot(veh,VehRoutDesPark, vehicle_info, current_dec, next_dec_list, visited_2, park_lot_2)

            # for current_dec, next_dec_list in routing_decisions_3.items():
            #     process_parking_lot(veh,VehRoutDesPark, vehicle_info, current_dec, next_dec_list, visited_3, park_lot_3)

            # for current_dec, next_dec_list in routing_decisions_4.items():
            #     process_parking_lot(veh,VehRoutDesPark, vehicle_info, current_dec, next_dec_list, visited_4, park_lot_4)

            # for current_dec, next_dec_list in routing_decisions_5.items():
            #     process_parking_lot(veh,VehRoutDesPark, vehicle_info, current_dec, next_dec_list, visited_5, park_lot_5)

        # for j in range(1,21):
        #     Vissim.Simulation.RunSingleStep()
        #     i += 1

In [502]:
i

5519

In [828]:
dec_and_lot = {103:[2590], 102:[2589], 101:[2588], 100:[2587], 99:[2586], 98:[2585, 2584]}

In [ ]:
# vehicle_info = {85: {'no': 0, 'park_dur': 0, 'destparklot': 0},
#                 94: {'no': 0, 'park_dur': 0},
#                 95: {'no': 0, 'park_dur': 0},
#                 120: {'no': 0, 'park_dur': 0},
#                 132: {'no': 0, 'park_dur': 0},
#                 143: {'no': 0, 'park_dur': 0},
#                 200: {'no': 0, 'park_dur': 0},
#                 201: {'no': 0, 'park_dur': 0},
#                 202: {'no': 0, 'park_dur': 0}
#                 }

In [821]:
def find_park_duration(vehicle_list, dec_no):
    for veh in vehicle_list:
        if veh.AttValue('RoutDecNo') not in [94,95,96,97,98,99,100,101,102,103]:
            continue
        next_dest = 2583 if veh in [94,95] else int(veh.AttValue('DestParkLot')) + 1
        # print(veh.AttValue('RoutDecNo'), veh.AttValue('ParkState'), veh.AttValue('DestParkLot'),  Vissim.Net.ParkingLots.ItemByKey(veh.AttValue('DestParkLot')).AttValue('CurAvail'))
        if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(next_dest).AttValue('CurAvail') == 1:
            if veh.AttValue('RoutDecNo') in [96,97,98,99,100,101,102,103]:
                
                Vissim.Net.Vehicles.ItemByKey(veh.AttValue('No')).SetAttValue('DwellTm', 0)


            # continue
            return veh.AttValue('No'), veh.AttValue('ParkDurCur')
    return 0, 0

In [ ]:
# Main simulation loop
for i in range(3000):
    vehicles = Vissim.Net.Vehicles
    count = 0

    if i % 50 == 0:
        veh_no, vehIn_94_ParkDur = find_park_duration(vehicles, 94)
        vehIn_95, vehIn_95_ParkDur = find_park_duration(vehicles, 95)
        print(veh_no, vehIn_94_ParkDur, vehIn_95, vehIn_95_ParkDur)

        if vehIn_94_ParkDur > vehIn_95_ParkDur:
            Vissim.Net.Vehicles.ItemByKey(veh_no).SetAttValue('DwellTm', 0)
        else :
            Vissim.Net.Vehicles.ItemByKey(vehIn_95).SetAttValue('DwellTm', 0)

        for i in  [94,95,96,97,98,99,100,101,102,103]:
            # if i in [96,97,98,99,100,101,102,103]:
            #     continue
            find_park_duration(vehicles, i)
            # if veh_no > 0:
            #     Vissim.Net.Vehicles.ItemByKey(veh_no).SetAttValue('DwellTm', 0)

    Vissim.Simulation.RunSingleStep()

In [823]:
veh_no, vehIn_94_ParkDur 

(0, 0)

In [826]:
def find_park_duration(vehicle_list, dec_no):
    for veh in vehicle_list:
        if veh.AttValue('RoutDecNo') == dec_no and veh.AttValue('ParkState') == 'ISPARKED' and Vissim.Net.ParkingLots.ItemByKey(veh.AttValue('DestParkLot')).AttValue('CurAvail') == 1:
            return veh.AttValue('No'), veh.AttValue('ParkDurCur')
    return 0, 0

def set_dwell_time(vehicles, veh_no, dwell_time):
    if veh_no != 0:
        vehicles.ItemByKey(veh_no).SetAttValue('DwellTm', dwell_time)

# Main simulation loop
for i in range(3000):
    vehicles = Vissim.Net.Vehicles
    count = 0

    if i % 250 == 0:
        veh_no, vehIn_94_ParkDur = find_park_duration(vehicles, 94)
        vehIn_95, vehIn_95_ParkDur = find_park_duration(vehicles, 95)
        veh_no_uber, park_dur_uber = find_park_duration(vehicles, 38)  # Adjust the route number

        if  Vissim.Net.ParkingLots.ItemByKey(veh.AttValue('DestParkLot')).AttValue('CurAvail') == 1 and (vehIn_94 != 0 or vehIn_95 != 0):
            if vehIn_94_ParkDur > vehIn_95_ParkDur:
                set_dwell_time(vehicles, vehIn_94, 0)
            elif vehIn_94_ParkDur < vehIn_95_ParkDur:
                set_dwell_time(vehicles, vehIn_95, 0)

            set_dwell_time(vehicles, veh_no_uber, 0)

            for j in range(1, 126):
                Vissim.Simulation.RunSingleStep()
                i += 1

        for dec in [98, 99, 100, 101, 102, 103]:
            veh_in_dec, _ = find_park_duration(vehicles, dec)
            if veh_in_dec != 0:
                set_dwell_time(vehicles, veh_in_dec, 0)
                continue

            if dec == 98:
                set_dwell_time(vehicles, vehIn_97_96, 0)
            else:
                set_dwell_time(vehicles, veh_in_dec, 0)

            # if vehIn_94_ParkDur > vehIn_95_ParkDur:
            #     set_dwell_time(vehicles, vehIn_94, 0)
            # elif vehIn_94_ParkDur < vehIn_95_ParkDur:
            #     set_dwell_time(vehicles, vehIn_95, 0)

            # set_dwell_time(vehicles, veh_no_uber, 0)

            for j in range(1, 126):
                Vissim.Simulation.RunSingleStep()
                i += 1

    Vissim.Simulation.RunSingleStep()


KeyboardInterrupt: 

# after initialization

In [825]:
for i in range(3000):
    pedestrian = Vissim.Net.Pedestrians
    vehicles = Vissim.Net.Vehicles
    count = 0
    if i%10 == 0:
        pedestrian = Vissim.Net.Pedestrians
        parkDur = 0
        veh_no = 0
        vehIn_95 = 0
        vehIn_95_ParkDur = 0
        vehIn_94_ParkDur = 0
        vehIn_94 = 0
        pedesNo = 0
        vehIn_103 = 0
        vehIn_102 = 0
        vehIn_101 = 0
        vehIn_100 = 0
        vehIn_99 = 0
        vehIn_98 = 0
        vehIn_97_96 = 0
        uber_reserved_veh = 0
        park_dur_uber = 0

        for pedes in pedestrian:
            if pedes.AttValue('StaRoutNo') == 5 and pedes.AttValue('PosInQueue') == 1 and pedes.AttValue("MotionState") == 'WAITINGATQUEUEHEAD':
                count += 1
                pedesNo = pedes.AttValue('No')
                # pedes.SetAttValue('DwellTm', 0)
                # print(pedes.AttValue('No'))
                # Vissim.Simulation.RunSingleStep()
                # i += 1
                break

        for veh in vehicles:
            if veh.AttValue('RoutDecNo') != None:
                if veh.AttValue('RoutDecNo') == 94 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_94 = veh.AttValue('No')
                    vehIn_94_ParkDur = veh.AttValue('ParkDurCur')

                if veh.AttValue('RoutDecNo') == 95 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_95 = veh.AttValue('No')
                    vehIn_95_ParkDur = veh.AttValue('ParkDurCur')
                
                if veh.AttValue('RoutDecNo') == 104 and veh.AttValue('ParkState') == 'ISPARKED':
                    veh_no = veh.AttValue('No')

                if veh.AttValue('RoutDecNo') == 103 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_103 = veh.AttValue('No')
                
                if veh.AttValue('RoutDecNo') == 102 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_102 = veh.AttValue('No')

                if veh.AttValue('RoutDecNo') == 101 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_101 = veh.AttValue('No')

                if veh.AttValue('RoutDecNo') == 100 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_100 = veh.AttValue('No')

                if veh.AttValue('RoutDecNo') == 99 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_99 = veh.AttValue('No')
                
                if veh.AttValue('RoutDecNo') == 98 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_98 = veh.AttValue('No')

                if (veh.AttValue('RoutDecNo') == 97 and veh.AttValue('ParkState') == 'ISPARKED') or  (veh.AttValue('RoutDecNo') == 96 and veh.AttValue('ParkState') == 'ISPARKED'):
                    vehIn_97_96 = veh.AttValue('No')

                if  (veh.AttValue('RoutDecNo') >= 38 and veh.AttValue('RoutDecNo') <= 46) and veh.AttValue('ParkState') == 'ISPARKED':
                    if veh.AttValue('ParkDurCur') > park_dur_uber:
                        park_dur_uber = veh.AttValue('ParkDurCur')
                        uber_reserved_veh = veh.AttValue('No')

        if vehIn_97_96 == 0 and (vehIn_94 !=0 or vehIn_95 != 0):
                # vehicles.ItemByKey(veh_no).SetAttValue('DwellTm', 0)
            if vehIn_94_ParkDur > vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_94).SetAttValue('DwellTm', 0)
            elif vehIn_94_ParkDur < vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_95).SetAttValue('DwellTm', 0)
            if uber_reserved_veh != 0:
                vehicles.ItemByKey(uber_reserved_veh).SetAttValue('DwellTm', 0)
            for j in range(1,126):
                Vissim.Simulation.RunSingleStep()
                i += 1

        if vehIn_98 == 0:
            if vehIn_97_96 != 0:
                vehicles.ItemByKey(vehIn_97_96).SetAttValue('DwellTm', 0)
            if vehIn_94_ParkDur > vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_94).SetAttValue('DwellTm', 0)
            elif vehIn_94_ParkDur < vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_95).SetAttValue('DwellTm', 0)
            if uber_reserved_veh != 0:
                vehicles.ItemByKey(uber_reserved_veh).SetAttValue('DwellTm', 0)
            for j in range(1,126):
                Vissim.Simulation.RunSingleStep()
                i += 1


        if  vehIn_99 == 0 :
            if vehIn_97_96 != 0:
                vehicles.ItemByKey(vehIn_97_96).SetAttValue('DwellTm', 0)
            if vehIn_98 != 0:
                vehicles.ItemByKey(vehIn_98).SetAttValue('DwellTm', 0)
            
            if vehIn_94_ParkDur > vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_94).SetAttValue('DwellTm', 0)
            elif vehIn_94_ParkDur < vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_95).SetAttValue('DwellTm', 0)
            if uber_reserved_veh != 0:
                vehicles.ItemByKey(uber_reserved_veh).SetAttValue('DwellTm', 0)
            for j in range(1,126):
                Vissim.Simulation.RunSingleStep()
                i += 1

        if vehIn_100 == 0 :
            if vehIn_97_96 != 0:
                vehicles.ItemByKey(vehIn_97_96).SetAttValue('DwellTm', 0)
            if vehIn_98 != 0:
                vehicles.ItemByKey(vehIn_98).SetAttValue('DwellTm', 0)
            if vehIn_99 != 0:
                vehicles.ItemByKey(vehIn_99).SetAttValue('DwellTm', 0)
            if vehIn_94_ParkDur > vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_94).SetAttValue('DwellTm', 0)
            elif vehIn_94_ParkDur < vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_95).SetAttValue('DwellTm', 0)
            if uber_reserved_veh != 0:
                vehicles.ItemByKey(uber_reserved_veh).SetAttValue('DwellTm', 0)
            for j in range(1,126):
                Vissim.Simulation.RunSingleStep()
                i += 1
  

        if vehIn_101 == 0 :
            if vehIn_97_96 != 0:
                vehicles.ItemByKey(vehIn_97_96).SetAttValue('DwellTm', 0)
            if vehIn_98 != 0:
                vehicles.ItemByKey(vehIn_98).SetAttValue('DwellTm', 0)
            if vehIn_99 != 0:
                vehicles.ItemByKey(vehIn_99).SetAttValue('DwellTm', 0)
            if vehIn_100 != 0:
                vehicles.ItemByKey(vehIn_100).SetAttValue('DwellTm', 0)
            if vehIn_94_ParkDur > vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_94).SetAttValue('DwellTm', 0)
            elif vehIn_94_ParkDur < vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_95).SetAttValue('DwellTm', 0)
            if uber_reserved_veh != 0:
                vehicles.ItemByKey(uber_reserved_veh).SetAttValue('DwellTm', 0)

            for j in range(1,126):
                Vissim.Simulation.RunSingleStep()
                i += 1

        if vehIn_102 == 0 :
            if vehIn_97_96 != 0:
                vehicles.ItemByKey(vehIn_97_96).SetAttValue('DwellTm', 0)
            if vehIn_98 != 0:
                vehicles.ItemByKey(vehIn_98).SetAttValue('DwellTm', 0)
            if vehIn_99 != 0:
                vehicles.ItemByKey(vehIn_99).SetAttValue('DwellTm', 0)
            if vehIn_100 != 0:
                vehicles.ItemByKey(vehIn_100).SetAttValue('DwellTm', 0)
            if vehIn_101 != 0:
                vehicles.ItemByKey(vehIn_101).SetAttValue('DwellTm', 0)

            if vehIn_94_ParkDur > vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_94).SetAttValue('DwellTm', 0)
            elif vehIn_94_ParkDur < vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_95).SetAttValue('DwellTm', 0)
            if uber_reserved_veh != 0:
                vehicles.ItemByKey(uber_reserved_veh).SetAttValue('DwellTm', 0)
            for j in range(1,126):
                Vissim.Simulation.RunSingleStep()
                i += 1
        

        if vehIn_103 == 0 :
            if vehIn_97_96 != 0:
                vehicles.ItemByKey(vehIn_97_96).SetAttValue('DwellTm', 0)
            if vehIn_98 != 0:
                vehicles.ItemByKey(vehIn_98).SetAttValue('DwellTm', 0)
            if vehIn_99 != 0:
                vehicles.ItemByKey(vehIn_99).SetAttValue('DwellTm', 0)
            if vehIn_100 != 0:
                vehicles.ItemByKey(vehIn_100).SetAttValue('DwellTm', 0)
            if vehIn_101 != 0:
                vehicles.ItemByKey(vehIn_101).SetAttValue('DwellTm', 0)
            if vehIn_102 != 0:
                vehicles.ItemByKey(vehIn_102).SetAttValue('DwellTm', 0)


            if vehIn_94_ParkDur > vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_94).SetAttValue('DwellTm', 0)
            elif vehIn_94_ParkDur < vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_95).SetAttValue('DwellTm', 0)
            if uber_reserved_veh != 0:
                vehicles.ItemByKey(uber_reserved_veh).SetAttValue('DwellTm', 0)

            for j in range(1,126):
                Vissim.Simulation.RunSingleStep()
                i += 1

        if veh_no == 0:
            if vehIn_97_96 != 0:
                vehicles.ItemByKey(vehIn_97_96).SetAttValue('DwellTm', 0)
            if vehIn_98 != 0:
                vehicles.ItemByKey(vehIn_98).SetAttValue('DwellTm', 0)
            if vehIn_99 != 0:
                vehicles.ItemByKey(vehIn_99).SetAttValue('DwellTm', 0)
            if vehIn_100 != 0:
                vehicles.ItemByKey(vehIn_100).SetAttValue('DwellTm', 0)
            if vehIn_101 != 0:
                vehicles.ItemByKey(vehIn_101).SetAttValue('DwellTm', 0)
            if vehIn_102 != 0:
                vehicles.ItemByKey(vehIn_102).SetAttValue('DwellTm', 0)
            if vehIn_103 != 0:
                vehicles.ItemByKey(vehIn_103).SetAttValue('DwellTm', 0)
            if vehIn_94_ParkDur > vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_94).SetAttValue('DwellTm', 0)
            elif vehIn_94_ParkDur < vehIn_95_ParkDur:
                vehicles.ItemByKey(vehIn_95).SetAttValue('DwellTm', 0)
            if uber_reserved_veh != 0:
                vehicles.ItemByKey(uber_reserved_veh).SetAttValue('DwellTm', 0)


            for j in range(1,126):
                Vissim.Simulation.RunSingleStep()
                i += 1

        for pedes in pedestrian:
            if pedes.AttValue('StaRoutNo') == 5 and pedes.AttValue('PosInQueue') == 1 and pedes.AttValue("MotionState") == 'WAITINGATQUEUEHEAD':
                pedesNo = pedes.AttValue('No')
                # pedes.SetAttValue('DwellTm', 0)
                # print(pedes.AttValue('No'))
                # Vissim.Simulation.RunSingleStep()
                # i += 1
                break

        for veh in vehicles:
            if veh.AttValue('RoutDecNo') != None:
                if veh.AttValue('RoutDecNo') == 94 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_94 = veh.AttValue('No')
                    vehIn_94_ParkDur = veh.AttValue('ParkDurCur')

                if veh.AttValue('RoutDecNo') == 95 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_95 = veh.AttValue('No')
                    vehIn_95_ParkDur = veh.AttValue('ParkDurCur')
                
                if veh.AttValue('RoutDecNo') == 104 and veh.AttValue('ParkState') == 'ISPARKED':
                    veh_no = veh.AttValue('No')

                if veh.AttValue('RoutDecNo') == 103 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_103 = veh.AttValue('No')
                
                if veh.AttValue('RoutDecNo') == 102 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_102 = veh.AttValue('No')

                if veh.AttValue('RoutDecNo') == 101 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_101 = veh.AttValue('No')

                if veh.AttValue('RoutDecNo') == 100 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_100 = veh.AttValue('No')

                if veh.AttValue('RoutDecNo') == 99 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_99 = veh.AttValue('No')
                
                if veh.AttValue('RoutDecNo') == 98 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_98 = veh.AttValue('No')

                if (veh.AttValue('RoutDecNo') == 97 and veh.AttValue('ParkState') == 'ISPARKED') or  (veh.AttValue('RoutDecNo') == 96 and veh.AttValue('ParkState') == 'ISPARKED'):
                    vehIn_97_96 = veh.AttValue('No')

                if  (veh.AttValue('RoutDecNo') >= 37 and veh.AttValue('RoutDecNo') <= 46) and veh.AttValue('ParkState') == 'ISPARKED':
                    if veh.AttValue('ParkDurCur') > park_dur_uber:
                        park_dur_uber = veh.AttValue('ParkDurCur')
                        uber_reserved_veh = veh.AttValue('No')

        if count > 0 and veh_no != 0:
       
            print(veh_no, pedesNo, vehIn_94, vehIn_95, vehIn_94_ParkDur, vehIn_95_ParkDur)


            if veh_no != 0 and pedesNo != 0:
                print("simulation time", Vissim.Simulation.AttValue('SimSec'))
                vehicles.ItemByKey(veh_no).SetAttValue('DwellTm', 0)


                if vehIn_103 != 0:
                    vehicles.ItemByKey(vehIn_103).SetAttValue('DwellTm', 0)
                # elif vehIn_103 == 0 and vehIn_102 == 0 and vehIn_101 == 0 and vehIn_100 == 0 and vehIn_99 == 0 and vehIn_98 == 0 and vehIn_27_26 == 0:
                #     VehRoutDesPark.ItemByKey(103).SetAttValue('ParkDur(1)',202)


                if vehIn_102 != 0:
                    vehicles.ItemByKey(vehIn_102).SetAttValue('DwellTm', 0)
                # elif vehIn_102 == 0 and vehIn_101 == 0 and vehIn_100 == 0 and vehIn_99 == 0 and vehIn_98 == 0 and vehIn_27_26 == 0:
                #     VehRoutDesPark.ItemByKey(102).SetAttValue('ParkDur(1)',202)


                if vehIn_101 != 0:
                    vehicles.ItemByKey(vehIn_101).SetAttValue('DwellTm', 0)
                # elif vehIn_101 == 0 and vehIn_100 == 0 and vehIn_99 == 0 and vehIn_98 == 0 and vehIn_27_26 == 0:
                #     VehRoutDesPark.ItemByKey(101).SetAttValue('ParkDur(1)',202)


                if vehIn_100 != 0:
                    vehicles.ItemByKey(vehIn_100).SetAttValue('DwellTm', 0)
                # elif vehIn_100 == 0 and vehIn_99 == 0 and vehIn_98 == 0 and vehIn_27_26 == 0:
                #     VehRoutDesPark.ItemByKey(100).SetAttValue('ParkDur(1)',202)


                if vehIn_99 != 0:
                    vehicles.ItemByKey(vehIn_99).SetAttValue('DwellTm', 0)
                # elif vehIn_99 == 0 and vehIn_98 == 0 and vehIn_27_26 == 0:
                #     VehRoutDesPark.ItemByKey(99).SetAttValue('ParkDur(1)',202)


                if vehIn_98 != 0:
                    vehicles.ItemByKey(vehIn_98).SetAttValue('DwellTm', 0)

                if vehIn_97_96 != 0:
                    vehicles.ItemByKey(vehIn_97_96).SetAttValue('DwellTm', 0)
                # else:
                #     print("waiting for car")
                


                if vehIn_94_ParkDur > vehIn_95_ParkDur:
                    vehicles.ItemByKey(vehIn_94).SetAttValue('DwellTm', 0)
                elif vehIn_94_ParkDur < vehIn_95_ParkDur:
                    vehicles.ItemByKey(vehIn_95).SetAttValue('DwellTm', 0)

                if uber_reserved_veh != 0:
                    vehicles.ItemByKey(uber_reserved_veh).SetAttValue('DwellTm', 0)
                # Vissim.Simulation.RunSingleStep()
                    
                pedestrian.ItemByKey(pedesNo).SetAttValue('DwellTm', 0)

                # i += 1
                count -= 1            

            # elif veh_no == 0 and pedesNo != 0:
            #     if uber_reserved_veh != 0:
            #         vehicles.ItemByKey(uber_reserved_veh).SetAttValue('DwellTm', 0)
            #     else:
            #         print("No car in reserved parking lot")
            
    Vissim.Simulation.RunSingleStep()

KeyboardInterrupt: 

In [643]:
veh_no = 51
pedesNo = 114


In [645]:
for i in range(3000):
    # if veh_no != 0 and pedesNo != 0:
    #     print("simulation time", Vissim.Simulation.AttValue('SimSec'))
    #     vehicles.ItemByKey(veh_no).SetAttValue('DwellTm', 0)


    #     pedestrian.ItemByKey(pedesNo).SetAttValue('DwellTm', 0)
    Vissim.Simulation.RunSingleStep()

KeyboardInterrupt: 

In [297]:
veh_no

0

In [243]:
pedestrian.ItemByKey(pedesNo)

<win32com.gen_py.Vissim Object Library 24.0 64 Bit.IPedestrian instance at 0x1592268963216>

In [277]:
Vissim.Net.VehicleRoutingDecisionsStatic.ItemByKey(26).VehRoutSta.ItemByKey(1).AttValue('RelFlow(1)')

0.8

# after initial seed

In [203]:
for i in range(2000):
    pedestrian = Vissim.Net.Pedestrians
    count = 0
    if i%100 == 0:
        for pedes in pedestrian:
                if pedes.AttValue('StaRoutNo') == 5 and (pedes.AttValue("MotionState") == 'WAITINGINQUEUE' or pedes.AttValue("MotionState") == 'WAITINGATQUEUEHEAD'):
                    count += 1

        while count > 0:
                # print(count)
            vehicles = Vissim.Net.Vehicles
            pedestrian = Vissim.Net.Pedestrians
            parkDur = 0
            veh_no = 0
            vehIn_95 = 0
            vehIn_95_ParkDur = 0
            vehIn_94_ParkDur = 0
            vehIn_94 = 0
            pedesNo = 0

            for pedes in pedestrian:
                if pedes.AttValue('StaRoutNo') == 5 and pedes.AttValue('PosInQueue') == 1 and pedes.AttValue("MotionState") == 'WAITINGATQUEUEHEAD':
                    pedesNo = pedes.AttValue('No')
                    # pedes.SetAttValue('DwellTm', 0)
                    # print(pedes.AttValue('No'))
                    # Vissim.Simulation.RunSingleStep()
                    # i += 1
                    break

            for veh in vehicles:
                if veh.AttValue('RoutDecNo') == 94 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_94 = veh.AttValue('No')
                    vehIn_94_ParkDur = veh.AttValue('ParkDurCur')

                if veh.AttValue('RoutDecNo') == 95 and veh.AttValue('ParkState') == 'ISPARKED':
                    vehIn_95 = veh.AttValue('No')
                    vehIn_95_ParkDur = veh.AttValue('ParkDurCur')
                
                if veh.AttValue('RoutDecNo') == 98 and veh.AttValue('ParkState') == 'ISPARKED':
                    veh_no = veh.AttValue('No')
                    
            print(veh_no, pedesNo, vehIn_94, vehIn_95, vehIn_94_ParkDur, vehIn_95_ParkDur)
            if veh_no != 0 and pedesNo != 0 :
                vehicles.ItemByKey(veh_no).SetAttValue('DwellTm', 0)
                vehicles.ItemByKey(pedesNo).SetAttValue('DwellTm', 0)
                if vehIn_94_ParkDur > vehIn_95_ParkDur:
                    vehicles.ItemByKey(vehIn_94).SetAttValue('DwellTm', 0)
                elif vehIn_94_ParkDur < vehIn_95_ParkDur:
                    vehicles.ItemByKey(vehIn_95).SetAttValue('DwellTm', 0)
                Vissim.Simulation.RunSingleStep()
                i += 1

                            
                    # for veh in vehicles:
                    #     if veh.AttValue('RoutDecNo') == 104 and veh.AttValue('ParkState') == 'ISPARKED':
                    #         veh_no = veh.AttValue('No')
                    #         vehicles.ItemByKey(veh_no).SetAttValue('DwellTm', 0)
                    #         if vehIn_94_ParkDur > vehIn_95_ParkDur:
                    #             vehicles.ItemByKey(vehIn_94).SetAttValue('DwellTm', 0)
                    #         elif vehIn_94_ParkDur < vehIn_95_ParkDur:
                    #             vehicles.ItemByKey(vehIn_95).SetAttValue('DwellTm', 0)
                                
                    #         Vissim.Simulation.RunSingleStep()
                    #         i += 1
                    # break
            count -= 1
    Vissim.Simulation.RunSingleStep()

KeyboardInterrupt: 

# Uber reserved zone

In [176]:
vehicles = Vissim.Net.Vehicles
ParkDurReserved = 0
veh_no_reserved = 0
for veh in vehicles:
    try:
        if veh.AttValue('RoutDecNo') > 35 and veh.AttValue('RoutDecNo') < 47:
            if veh.AttValue('ParkDurCur') > ParkDurReserved:
                ParkDurReserved = veh.AttValue('ParkDurCur')
                veh_no_reserved = veh.AttValue('No')
                print(veh.AttValue('No'), veh.AttValue('ParkDurCur'))
    except:
        pass
vehicles.ItemByKey(veh_no_reserved).SetAttValue('DwellTm', 0)


        # print(veh.AttValue('No'), veh.AttValue('ParkDurCur'))
    # if veh.AttValue('RoutDecNo') > 35 and veh.AttValue('RoutDecNo') < 47:
    #     if veh.AttValue('ParkDurCur') > ParkDurReserved:
    #         ParkDurReserved = veh.AttValue('ParkDurCur')
    #         print(veh.AttValue('No'), veh.AttValue('ParkDurCur'))
        # print(veh.AttValue('No'), veh.AttValue('ParkDurCur'))
    # print(veh.AttValue('No'), veh.AttValue('ParkDurCur'))

2 218.89999999999185
7 230.1999999999912


In [156]:
for veh in vehicles:
    # print(veh.AttValue('No'), veh.AttValue('VehType'),veh.AttValue('ParkDurCur'))
    # print(veh.AttValue('Lane')
    if veh.AttValue('Lane').startswith('41'):
        print(veh.AttValue('ParkDurCur'))

541.4000000000545
None
509.50000000004735
520.8000000000499
503.9000000000461
496.4000000000444
501.0000000000454
482.20000000004114
479.4000000000405
462.80000000003673
420.60000000002714
412.0000000000252
390.6000000000203
380.0000000000179
383.0000000000186
317.2000000000036
292.59999999999803
None
277.69999999999465
282.09999999999565
245.29999999999035
None


In [150]:
if veh.AttValue('ParkDurCur') > parkDur:
                            parkDur = veh.AttValue('ParkDurCur')
                            veh_no = veh.AttValue('No')
                            print(veh_no)
                            print(parkDur)

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [135]:
pedestrian = Vissim.Net.Pedestrians
for pedes in pedestrian:
    # print(pedes.AttValue("Name"))
    print(pedes.AttValue("StaRoutNo"))
    print(pedes.AttValue("MotionState"))
    print(pedes.AttValue("DwellTm"))

    # print(pedes.AttValue("StaRoutNo").)

3
WALKINGONLEVEL
None
1
WALKINGONLEVEL
None
3
WALKINGONLEVEL
None
3
WALKINGONLEVEL
None
6
WALKINGONLEVEL
None
3
WALKINGONLEVEL
None
6
WAITINGINQUEUE
None
5
WAITINGINQUEUE
None
3
WALKINGONLEVEL
None
6
WAITINGINQUEUE
None
3
WALKINGONLEVEL
None
1
WALKINGONLEVEL
None
1
WALKINGONLEVEL
None
4
WAITINGINQUEUE
None
3
WALKINGONLEVEL
None
1
WALKINGONLEVEL
None
3
WALKINGONLEVEL
None
6
WAITINGINQUEUE
None
3
WALKINGONLEVEL
None
4
WAITINGINQUEUE
None
3
WALKINGONLEVEL
None
1
WALKINGONLEVEL
None
6
WAITINGINQUEUE
None
6
WAITINGINQUEUE
None
5
WAITINGINQUEUE
None
5
WALKINGONLEVEL
None
1
WALKINGONLEVEL
None
3
WALKINGONLEVEL
None
5
WALKINGONLEVEL
None
1
WALKINGONLEVEL
None
4
WAITINGINQUEUE
None
5
WAITINGINQUEUE
None
1
WALKINGONLEVEL
None
5
WAITINGINQUEUE
None
4
WAITINGINQUEUE
None
7
WAITINGATQUEUEHEAD
13.200000000000001
1
WAITINGATQUEUEHEAD
12.950000000000001
1
WALKINGONLEVEL
None
5
WAITINGINQUEUE
None
1
WALKINGONLEVEL
None
1
WALKINGONLEVEL
None
3
WALKINGONLEVEL
None
5
WAITINGATQUEUEHEAD
5.3500000000000005


# Simulation before 10 minutes

In [ ]:
print(f'Number of vehicles to be supplied initially to reserved parking lot is {initial_volume} and for pickup zones is {initial_volume_1} per hour for 10 minutes')
# number_of_executions = 0
Sim_break_at = 60 # simulation second [s]
Vissim.Simulation.SetAttValue('SimBreakAt', Sim_break_at)
Vissim.Simulation.RunContinuous() 

simulation_second = Vissim.Net.Simulation.AttValue('SimSec')
group_parking_lot_attributes = Vissim.Net.ParkLotGrps.GetMultipleAttributes(( 'Name','No','NumVehsEnter(Current,Total,All)',
                                                                             'NumVehsEnter(Current,Last,All)',
                                                                             'NumVehsPark(Current,Last,All)',
                                                                             'NumVehsLeave(Current,Total,All)',
                                                                             'OccupDurTot(Current,Last,All)',
                                                                             'OccupRate(Current,Last,All)'))

for i in range(12,15):
    # print('Name: ', group_parking_lot_attributes[i][0], 'No: ', group_parking_lot_attributes[i][1],
    #        'NumVehsEnter(Current,Total,All): ', group_parking_lot_attributes[i][2], 'NumVehsEnter(Current,Last,All): ',
    #          group_parking_lot_attributes[i][3], 'NumVehsPark(Current,Total,All): ', group_parking_lot_attributes[i][4],
    #            'NumVehsLeave(Current,Total,All): ', group_parking_lot_attributes[i][5], 'OccupDurTot(Current,Last,All): ',
    #              group_parking_lot_attributes[i][6], 'OccupRate(Current,Last,All): ', group_parking_lot_attributes[i][7])
    print(f'Number of vehicles in the parking lot {group_parking_lot_attributes[i][0]} is {group_parking_lot_attributes[i][2]} and the occupancy rate is {group_parking_lot_attributes[i][7]}')

for i in range(1,11):
    Sim_break = Sim_break_at + 60
    Vissim.Simulation.SetAttValue('SimBreakAt', Sim_break)
    Vissim.Simulation.RunContinuous()

    ParkingLots = Vissim.Net.ParkingLots
    keys = [950]
    keys_stage_area = []
    for key in keys:
        Avail = ParkingLots.ItemByKey(key).AttValue('CurAvail')
        print(f'Number of vehicles in the parking lot {ParkingLots.ItemByKey(key).AttValue("Name")} is {Avail}')

    VehRoutDesPark = Vissim.Net.VehicleRoutingDecisionsParking
    keys_stage_area_park_dec = [94,95]
    if Avail > 0:
        for key in keys_stage_area_park_dec:
            decision = VehRoutDesPark.ItemByKey(key)
            decision.SetAttValue('ParkDur(1)',1)
            # decision.SetAttValue('FullOccupBehav', 'DriveOn')
            print(f'Parking lot {key} is available')
    else:
        for key in keys_stage_area_park_dec:
            decision = VehRoutDesPark.ItemByKey(key)
            decision.SetAttValue('ParkDur(1)',201)
            # decision.SetAttValue('FullOccupBehav', 'Wait')
            print(f'Parking lot {key} is full')

    Sim_break_at += 60
    




# for count in range(0, len(group_parking_lot_attributes[0])):
#     if group_parking_lot_attributes[count][0] == 'ola_reserves_1':
#         Name.append(group_parking_lot_attributes[count][0])
#         No.append(group_parking_lot_attributes[count][1])
#         NumVehsEnter.append(group_parking_lot_attributes[count][2])
    # NumVehsLeave.append(Vissim.Net.ParkingLots.ItemByKey(group_parking_lot_attributes[count][1]).AttValue('NumVehsLeave(Current,Current,All)'))

# print("Name", Name)
# print("No", No)
# print("NumVehsEnter", NumVehsEnter)
# print("simulation second", simulation_second)

# Simulation after initial 10 minutes

In [10]:
set_pedestrian_prev = set()
deque_pedestrian_curr = deque()
break_time = 605
for i in range(1, 17281):
    Sim_break_at = break_time # simulation second [s]
    # if Sim_break_at != 360:
    # print("break time", break_time)
    Vissim.Simulation.SetAttValue('SimBreakAt', Sim_break_at)
    Vissim.Simulation.RunContinuous()
    for Ped in Vissim.Net.Pedestrians:
        if Ped.AttValue('No') not in set_pedestrian_prev:
            deque_pedestrian_curr.append(Ped.AttValue('No'))
        set_pedestrian_prev.add(Ped.AttValue('No'))
    # print("set pedestrian prev", set_pedestrian_prev)

    for Vehicle in Vissim.Net.Vehicles:
        num_pedestrian = len(deque_pedestrian_curr)
        veh_rout_park = Vehicle.AttValue('VehRoutPark')
        if veh_rout_park is not None:
            if veh_rout_park.startswith('18') or veh_rout_park.startswith('20') or veh_rout_park.startswith('22'):
                if num_pedestrian > 0 and Vehicle.AttValue('DwellTm') != 0:
                    Vehicle.SetAttValue('DwellTm', 0)
                    num_pedestrian -= 1
                    removed_pedestrian = deque_pedestrian_curr.popleft()  # Remove the first element
                # print("Removed pedestrian:", removed_pedestrian)
                # print("Remaining pedestrians:", len(list(deque_pedestrian_curr)))
                # print("Remaining pedestrians list:", list(deque_pedestrian_curr))

    simulation_second = Vissim.Net.Simulation.AttValue('SimSec')
    group_parking_lot_attributes = Vissim.Net.ParkLotGrps.GetMultipleAttributes(( 'Name','No','NumVehsEnter(Current,Total,All)',
                                                                             'NumVehsEnter(Current,Last,All)',
                                                                             'NumVehsPark(Current,Last,All)',
                                                                             'NumVehsLeave(Current,Last,All)',
                                                                             'OccupDurTot(Current,Last,All)',
                                                                             'OccupRate(Current,Last,All)'))
    for number in range(12,15):
        # print('Name: ', group_parking_lot_attributes[number][0], 'No: ', group_parking_lot_attributes[number][1],
        #     'NumVehsEnter(Current,Total,All): ', group_parking_lot_attributes[number][2], 'NumVehsEnter(Current,Last,All): ',
        #         group_parking_lot_attributes[number][3], 'NumVehsPark(Current,Total,All): ', group_parking_lot_attributes[number][4],
        #         'NumVehsLeave(Current,Total,All): ', group_parking_lot_attributes[number][5], 'OccupDurTot(Current,Last,All): ',
        #             group_parking_lot_attributes[number][6], 'OccupRate(Current,Last,All): ', group_parking_lot_attributes[number][7])
        print(f'PARKING LOT : {group_parking_lot_attributes[number][0]}')
        print(f'Number of vehicles entering: {group_parking_lot_attributes[number][2]} Number of vehicles leaving: {group_parking_lot_attributes[number][5]} Number of vehicles parked: {group_parking_lot_attributes[number][4]} Occupancy rate: {group_parking_lot_attributes[number][7]}')
    # for count in range(0, len(group_parking_lot_attributes[0])):
    #     Name.append(group_parking_lot_attributes[count][0])
    #     No.append(group_parking_lot_attributes[count][1])
    #     NumVehsEnter.append(group_parking_lot_attributes[count][2])
        # NumVehsLeave.append(Vissim.Net.ParkingLots.ItemByKey(group_parking_lot_attributes[count][1]).AttValue('NumVehsLeave(Current,Current,All)'))
    # print("Name", Name)
    # # print("No", No)
    # print("NumVehsEnter", NumVehsEnter)
    print("simulation second", simulation_second)
    print()
    print()
    print('----------------------------------------------')
    # if break_time != 360:
        # break_time = 120
    # if break_time == 120:
    #     break
    break_time = break_time + 5
Vissim.Simulation.RunContinuous()

PARKING LOT : ola_reserves_1
Number of vehicles entering: 29 Number of vehicles leaving: 0 Number of vehicles parked: 29 Occupancy rate: 0.24058479532157878
PARKING LOT : ola_reserves_2
Number of vehicles entering: 34 Number of vehicles leaving: 0 Number of vehicles parked: 34 Occupancy rate: 0.27700292397651644
PARKING LOT : ola_reserves_3
Number of vehicles entering: 31 Number of vehicles leaving: 0 Number of vehicles parked: 31 Occupancy rate: 0.2615804597700344
simulation second 605.0


----------------------------------------------
PARKING LOT : ola_reserves_1
Number of vehicles entering: 29 Number of vehicles leaving: 0 Number of vehicles parked: 29 Occupancy rate: 0.24058479532157878
PARKING LOT : ola_reserves_2
Number of vehicles entering: 34 Number of vehicles leaving: 0 Number of vehicles parked: 34 Occupancy rate: 0.27700292397651644
PARKING LOT : ola_reserves_3
Number of vehicles entering: 31 Number of vehicles leaving: 0 Number of vehicles parked: 31 Occupancy rate: 0.2615

KeyboardInterrupt: 